In [1]:
import pickle
import pandas as pd
import re


with open('data_pickle_format/synethetictotrain.pickle', 'rb') as f:
    synethetic = pickle.load(f)
with open('data_pickle_format/labeledtotrain.pickle', 'rb') as f:
    labeled = pickle.load(f)
with open('data_pickle_format/unlabeled_vacancies_final.pickle', 'rb') as f:
    unlabeled = pickle.load(f)

In [2]:
# labeled

In [3]:
# synethetic

In [4]:
# unlabeled

In [5]:
from string import punctuation
import html
import re

def preprocess_text(text):
    text = text.lower()
    text = html.unescape(text)
    text = re.sub(r"https?://\S+", "", text) # remove http links
    text = re.sub(r"<.*?>", " ", text) # remove html tags
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)  # Remove punctuation
    text = " ".join(text.split())  # Remove extra spaces, tabs, and new lines
    return text

unlabeled["job_description"] = unlabeled["job_description"].map(preprocess_text)

pd.set_option('display.max_colwidth', None)

In [6]:
unlabeled

positiontitle  \
0                  Freelance Technisch Consultant   
1                            Technisch Consultant   
2                                Support Engineer   
3                           Zelfstandig Architect   
4    Java Solution Architect / Software Architect   
..                                            ...   
824                Logistiek SAP Consultant Cloud   
825                        Functionele Consultant   
832                    Functioneel Consultant SAP   
833                    SAP Functioneel Consultant   
834                    Functioneel Consultant SAP   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [7]:
training = pd.concat([synethetic, labeled], axis=0)

In [8]:
training

,positiontitle,job_description,women_proportion
0,IT Support Technician,"we are looking for a helpful and knowledgeable it support technician to be the first point of contact for technical assistance. this role requires a balance of independent problem-solving and excellent interpersonal skills. you will be responsible for diagnosing and resolving hardware, software, and networking issues for our employees with efficiency and patience. you will have the autonomy to manage support tickets and prioritize tasks to ensure timely resolutions. your technical expertise and analytical approach will be key to your success. at the same time, this is a highly collaborative role where you will interact with employees across all departments. strong communication and empathy are crucial for providing outstanding customer service and support. our team is committed to creating a positive and helpful work environment, and we support your professional growth through training and development opportunities. we are seeking a candidate who is a self-motivated problem-solver and a friendly, approachable team member. your work ensures that our entire organization can operate smoothly and efficiently. requirements: associate's or bachelor's degree in information technology or a related field. relevant certifications (e.g., comptia a+, microsoft certified) are a plus. proven experience as an it support technician or in a similar customer-facing technical role. strong understanding of computer systems, mobile devices, and other tech products. excellent problem-solving and communication skills. an ability to work independently to resolve technical issues while maintaining a supportive and collaborative attitude. a customer-oriented mindset and a passion for helping others.",0.552450
1,Data Scientist,"we are looking for a curious and analytical data scientist to extract powerful insights from our data. this role provides the autonomy to explore complex datasets, develop predictive models, and drive data-driven decision-making across the company. you must be a highly motivated, independent thinker with a strong quantitative background, comfortable working in a fast-paced environment where your findings have a direct impact on business strategy. this individual focus is paired with a collaborative team culture. you will work with engineers, product managers, and business leaders in a supportive setting that values clear communication and the sharing of knowledge. we believe that empathy and diverse perspectives lead to better insights. we are invested in your personal development, providing opportunities to attend conferences and work on groundbreaking projects. our commitment to work-life balance ensures you have the space to think deeply and creatively. your work will help us solve challenging problems and contribute to our social mission. requirements: master's or phd in statistics, computer science, mathematics, or a related quantitative field. proven experience as a data scientist, with a strong portfolio of data science projects. excellent analytical and problem-solving skills, with a high degree of intellectual curiosity and independence. proficiency in python or r, and experience with machine learning frameworks like scikit-learn or tensorflow. strong communication and collaboration skills, with the ability to explain complex concepts to non-technical audiences. a competitive desire to uncover transformative insights, combined with a supportive and team-oriented spirit. a passion for learning and applying new data science techniques.",0.528720
2,Information Security Manager,"we are looking for a driven and strategic information security manager to lead our security operations and governance functions. this is a leadership role for a focused professional who can take ownership of our overall security posture and drive a culture of security awareness. you will have the autonomy to manage the security team, oversee risk assessments, and direct incident

In [9]:
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

train_df = training[["job_description", "women_proportion"]].dropna()
train_df.columns = ["text", "labels"]

from sklearn.model_selection import train_test_split
train_df, dev_df = train_test_split(train_df, test_size=0.1, random_state=42)


/home/jkoz/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import AutoModel, AutoTokenizer

model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

print("Model and tokenizer loaded successfully")

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()
model_args.num_train_epochs = 4
model_args.evaluate_during_training = True
model_args.regression = True   
model = ClassificationModel(
    "bert", 
    "bert-base-multilingual-cased", 
    num_labels=1, 
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

model.train_model(
    train_df[['text', 'labels']], 
    eval_df=dev_df[['text', 'labels']]
)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
5it [00:01,  3.35it/s]                       
Epoch 1 of 4:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model.train_model(train_df, eval_df=dev_df)

In [ ]:

preds, raw_outputs = model.predict(unlabeled['job_description'].tolist())

In [ ]:
# unlabeled_df = unlabeled[["job_description"]].dropna()
# unlabeled_df.columns = ["text"]

# # Predict
# predictions, _ = model.predict(unlabeled_df["text"].tolist())

# unlabeled["predicted_women_proportion"] = predictions


In [ ]:
# unlabeled